# The forward and backward passes 

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_01 import *
from pdb import set_trace

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

def normalize(x, mean, std):
    return (x-mean)/std

In [3]:
x_train, y_train, x_valid, y_valid = get_data()

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(-6.2598e-06), tensor(1.))

In [7]:
#export
def test_near_zero(x, tol=1e-3):
    assert x.abs()<tol, f"Near zero: {x}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(x_train.std() - 1)

In [9]:
n, m = x_train.shape
c = y_train.max() + 1
n, m, c

(50000, 784, tensor(10))

## Model with one hidden layer *from the foundations*

### Basic architecture

In [10]:
nh = 50

The next cell uses a simplified version of Kaiming initialization/He initialization. In case anybody is reading this: see [ebook by Michael Nielson](http://neuralnetworksanddeeplearning.com/chap3.html) for a detailed explanation.

In [11]:
w1 = torch.randn(m, nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [12]:
test_near_zero(w1.mean())
test_near_zero(w1.std() - 1/math.sqrt(m))

In [13]:
def lin(x, w, b):
    return x@w + b

In [14]:
t = lin(x_valid, w1, b1)

In [15]:
t.mean(), t.std()

(tensor(-0.0176), tensor(1.0103))

**Thanks to our initialization the mean and std are approximately equal to 0, 1!**

In [16]:
def relu(x):
    return x.clamp_min(0.)

In [17]:
t = relu(lin(x_valid, w1, b1))

In [18]:
t.mean(), t.std()

(tensor(0.3880), tensor(0.5738))

**Unfortunately not mean 0, std 1 anymore. That makes sense, however, because we removed all negative activations!**

**We have to change our initialization in order to account for the rectifiers! This is where He init comes into play.**

In [19]:
w1 = torch.randn(m, nh)*math.sqrt(2/m)

In [20]:
w1.mean(), w1.std() - math.sqrt(2/m)

(tensor(-0.0003), tensor(-7.0754e-05))

In [21]:
t = relu(lin(x_valid, w1, b1))

In [22]:
t.mean(), t.std()

(tensor(0.5773), tensor(0.8082))

In [23]:
#export
from torch.nn import init

In [24]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [25]:
t.mean(), t.std()

(tensor(0.5250), tensor(0.7648))

In [26]:
init.kaiming_normal_??

From the documentation:

mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in the
            forward pass. Choosing ``'fan_out'`` preserves the magnitudes in the
            backwards pass.
            
Why did we use `fan_out` here instead of `fan_in` despite dividing by `m` beforehand?            

In [27]:
w1.shape

torch.Size([784, 50])

In [28]:
torch.nn.Linear(m, nh).weight.shape

torch.Size([50, 784])

**For a linear layer PyTorch uses the transposed weight matrix. This is why we use `fan_out` instead of `fan_in`.**

In [29]:
torch.nn.Linear.forward??

In [30]:
torch.nn.functional.linear??

**The standard deviation is now approx 1. Can we reduce the mean to 0?**

In [31]:
def relu(x): return x.clamp_min(0.) - 0.5

In [32]:
w1 = torch.randn(m, nh)*math.sqrt(2./m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t1.std()

(tensor(0.0271), tensor(0.8175))

**Closer to mean 0, std 1!**

In [33]:
def model(x):
    l1 = lin(x, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [34]:
%timeit -n 10 _ = model(x_valid)

5.41 ms ± 81.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0], 1])

### Loss function: MSE (only for demo purposes)

In [36]:
model(x_valid).squeeze(-1).shape

torch.Size([10000])

In [37]:
#export
def mse(output, target):
    return (output.squeeze(-1) - target).pow(2).mean()

In [38]:
y_train, y_valid = y_train.float(), y_valid.float()

In [39]:
preds = model(x_valid)

In [40]:
preds.shape

torch.Size([10000, 1])

In [41]:
mse(preds, y_valid)

tensor(25.2947)

### Gradients and the backward pass

In [42]:
def mse_grad(output, target):
    # gradient of loss function with respect to activations of output layer
    output.g = 2. * (output.squeeze(-1) - target).unsqueeze(-1) / output.shape[0]

In [43]:
mse_grad(preds, y_valid)
preds.g.shape

torch.Size([10000, 1])

In [44]:
def relu_grad(inp, outp):
    # gradient of relu with respect to z = w*x + b
    inp.g = (inp>0).float() * outp.g

In [45]:
def lin_grad(inp, outp, w, b):
    # gradient of matrix multiplication with respect to input
    # Example: w has the shape torch.Size([50, 1]), b the shape torch.Size([1])
    #set_trace()
    inp.g = outp.g @ w.t()
    w.g = (inp.unsqueeze(-1) * outp.g.unsqueeze(1)).sum(0)  # shapes: [10000, 50, 1] x [10000, 1, 1] -> [50, 1]
    b.g = outp.g.sum(0)  # torch.Size([1])
    
    # the sum is over the batch dimension. Divide by n in function mse_grad.

In [46]:
def forward_and_backward(inp, targ):
    # forward pass through the network
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    
    loss = mse(out, targ)  # not actually needed for backward pass
    
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [47]:
forward_and_backward(x_valid, y_valid)

**Let's make sure the gradients are correct by using PyTorch's autograd:**

First, save our results:

In [48]:
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
inpg = x_valid.g.clone()

Calculate the gradients automatically with PyTorch:

In [49]:
x_valid2 = x_valid.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [50]:
def forward(inp, targ):
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    
    loss = mse(out, targ)
    
    return loss

In [51]:
loss = forward(x_valid2, y_valid)

In [52]:
loss.backward()

In [53]:
test_near(w12.grad, w1g)
test_near(w22.grad, w2g)
test_near(b12.grad, b1g)
test_near(b22.grad, b2g)
test_near(x_valid2.grad, inpg)

### Let's refactor the model: Layers as classes

**The `__call__` makes the classes functors.**

In [54]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.) - 0.5
        return self.out
        
    def backward(self):
        self.inp.g = (self.inp>0).float() * self.out.g

In [55]:
class Lin():
    def __init__(self, w, b):
        self.w = w
        self.b = b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)
        
    # Note: the huge tensor product/outer product in order to sum is pretty inefficient.

In [56]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2 * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [57]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()

In [58]:
w1.g, b1.g, w2.g, b2.g = [None] * 4

In [59]:
model = Model(w1, b1, w2, b2)

In [60]:
%time loss = model(x_valid, y_valid)

CPU times: user 29.2 ms, sys: 0 ns, total: 29.2 ms
Wall time: 6.2 ms


In [61]:
%time model.backward()

CPU times: user 828 ms, sys: 481 ms, total: 1.31 s
Wall time: 308 ms


In [62]:
test_near(w12.grad, w1.g)
test_near(w22.grad, w2.g)
test_near(b12.grad, b1.g)
test_near(b22.grad, b2.g)
test_near(x_valid2.grad, x_valid.g)

**There is lot's of duplicate code because every `__call__` method sets `self.inp` and `self.out`. Let's introduce the PyTorch Module.**

In [63]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented!')
    def backward(self): self.bwd(self.out, *self.args)

In [64]:
class Relu(Module):
    def forward(self, inp):
        return inp.clamp_min(0.) - 0.5
        
    def bwd(self, out, inp):
        inp.g = (inp>0).float() * out.g

In [81]:
class Lin(Module):
    def __init__(self, w, b):
        self.w = w
        self.b = b
        
    def forward(self, inp):
        return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        #self.w.g = torch.einsum('bi,bj->ij', inp, out.g)  # [10000, 50] x [10000, 1] -> [50, 1]  => 'bi,bj->ij'
        self.w.g = inp.t() @ out.g  # equivalent
        self.b.g = self.out.g.sum(0)

In [82]:
class Mse(Module):
    def forward(self, inp, targ):
        return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ):
        inp.g = 2 * (inp.squeeze() - targ).unsqueeze(-1) / targ.shape[0]

In [83]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()

In [84]:
w1.g, b1.g, w2.g, b2.g = [None] * 4

In [85]:
model = Model()

In [86]:
%time loss = model(x_valid, y_valid)

CPU times: user 51.4 ms, sys: 0 ns, total: 51.4 ms
Wall time: 9.67 ms


In [87]:
%time model.backward()

CPU times: user 104 ms, sys: 0 ns, total: 104 ms
Wall time: 18.6 ms


In [88]:
test_near(w12.grad, w1.g)
test_near(w22.grad, w2.g)
test_near(b12.grad, b1.g)
test_near(b22.grad, b2.g)
test_near(x_valid2.grad, x_valid.g)

## nn.Linear and nn.Module

In [89]:
#export
from torch import nn

In [90]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x.squeeze(), targ)

In [91]:
model = Model(m, nh, 1)

In [101]:
%time loss = model(x_valid, y_valid)

CPU times: user 46.9 ms, sys: 0 ns, total: 46.9 ms
Wall time: 10.2 ms


In [102]:
%time loss.backward()

CPU times: user 61.3 ms, sys: 2.18 ms, total: 63.5 ms
Wall time: 12.9 ms


## Export

In [104]:
!python notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py
